#### Gram-Schmidt procedure

Suppose $A=\begin{bmatrix}a_1 & a_2 & \cdots & a_k\end{bmatrix}\in \mathbf{R}^{n \times k}$ is `full rank`, recall the classic G-S (CGS) procedure wants to find the $Q$ and $R$ matrices such that

$$\begin{bmatrix}a_1 & a_2 & \cdots & a_k\end{bmatrix}=\begin{bmatrix}q_1 & q_2 & \cdots & q_k\end{bmatrix}\begin{bmatrix}r_{11} & r_{12} & \cdots & r_{1k} \\ 0 & r_{22} & \cdots & r_{2k} \\ \vdots & \vdots & \ddots & \vdots \\ 0 & 0& \cdots & r_{kk} \end{bmatrix}$$


$$a_i = (q_1^Ta_i)q_1 + (q_2^Ta_i)q_2 +\cdots+ (q_{i-1}^Ta_i)q_{i-1} + \|\tilde{q_i}\|q_i$$

We write the expression in the form of `sum of coefficients times vectors`

$$\begin{align*}
\tilde{q_i} &= a_i - (q_1^Ta_i)q_1 - (q_2^Ta_i)q_2 -\cdots- (q_{i-1}^Ta_i)q_{i-1}, q_i = \frac{\tilde{q_i}}{\|\tilde{q_i}\|}
\end{align*}$$

and we sequentially compute each term $(q_1^Ta_i)q_1$ and `subtract` it from $a_i$ to get $q_i$ (after normalization)

#### Equivalent iterative procedure

On the other hand, we know that we can also write the expression in the form of `rank-one projection matrix`

$$\begin{align*}
\tilde{q_i} &= a_i - (q_1q_1^T)a_i - (q_2q_2^T)a_i -\cdots- (q_{i-1}q_{i-1})^Ta_i\\
&=(I-Q_{i-1}Q_{i-1}^T)a_i \\
&=(I-q_{i-1}q_{i-1}^T)(I-q_{i-2}q_{i-2}^T)\cdots (I-q_1q_1^T) a_i
\end{align*}$$

where $Q_{i-1}=\begin{bmatrix}q_1 & q_2 & \cdots & q_{i-1}\end{bmatrix}$

The equality

$$(I-Q_{i-1}Q_{i-1}^T)a_i =(I-q_{i-1}q_{i-1}^T)(I-q_{i-2}q_{i-2}^T)\cdots (I-q_1q_1^T) a_i$$

follows from the fact that when two unit vectors $u, v$ are orthogonal, the projection onto the complement of $R([u, v])$ is equivalent to projection first onto the complement of $R(u)$ and then to the complement of $R(v)$

$$(I-vv^T)(I-uu^T)=I-uu^T-vv^T+u(u^Tv)v^T=I-vv^T-uu^T=(I-[u, v][u,v]^T)$$

This also indicates that at every iteration when we subtract $-q_{j}q_{j}^Ta_i$, it is equivalent to left multiplying $(I-q_{j}q_{j}^T)$

#### Modified G-S (MGS)

With this formulation, the MGS utilizes the following iterative procedure which allows to use prior result of $q_i$ in computation rather than always using $a_i$

$$\begin{align*}
q_i^{(1)}&=a_i \\
q_i^{(2)}&=(I-q_1q_1^T) a_i= a_i-(q_1^Ta_i)q_1 \,\,\text{(same as CGS)} \\
q_i^{(3)}&=(I-q_2q_2^T)q_i^{(2)}=q_i^{(2)}-\left(q_2^Tq_i^{(2)}\right)q_2 \,\,\text{(different from CGS)} \\
q_i^{(4)}&=(I-q_3q_3^T)q_i^{(3)}=q_i^{(3)}-\left(q_3^Tq_i^{(3)}\right)q_3 \,\,\text{(different from CGS)} \\
&\vdots \\
q_i^{(i)}&=(I-q_{i-1}q_{i-1}^T)q_i^{({i-1})}=q_i^{({i-1})}-\left(q_{i-1}^Tq_i^{({i-1})}\right)q_{i-1} \,\,\text{(different from CGS)}
\end{align*}$$

In the algorithm, we need to revise `R[j, i] = np.dot(Q[:, j], A[:, i])` to `R[j, i] = np.dot(Q[:, j], q)`

#### Intuition behind numerical stability of MGS

The orthogonalization coefficient computation in MGS $q_j^Tq_i^{(k)}$ only allows to propagate the potential error in $q_j$, which may span high dimension, to be restricted to $q_i^{(k)}$ and the dimension it occupies is getting lower as iteration $k$ moves on. Therefore, error propagation is getting limited


However, in CGS, the coefficient computation always involves $a_i$ (which is in high dimension) in $q_j^Ta_i$, therefore, error in $q_j$ won't be restricted to a lower dimension

#### Applicability

General MGS works with tall, square and fat matrices, and full rank and rank deficient matrices

#### Example

In [1]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [2]:
def general_gram_schmidt(A, modified=True):
    _, k = A.shape  # Get number of vectors (columns) in A
    Q = []  # Start with empty list, as we don't know how many q's are there
    R = np.zeros((0, k))  # Same here

    for i in range(k):
        # Loop over all a_i
        q = A[:, i].copy()

        # This skips when i=0
        for j in range(len(Q)):
            if modified:
                R[j, i] = np.dot(Q[j], q)
            else:
                R[j, i] = np.dot(Q[j], A[:, i])
            q -= R[j, i] * Q[j]

        # Compute norm of new q
        norm_q = np.sqrt(np.dot(q, q))

        # Only add q to Q if it is not small
        if norm_q > 1e-10:  # Tolerance
            q /= norm_q
            Q.append(q)

            # Expand R to include new row corresponding to new q
            new_row = np.zeros((1, k))
            new_row[0, i] = norm_q
            R = np.vstack([R, new_row])

    Q = np.column_stack(Q)  # Convert to array

    return Q, R

In [13]:
mat_list = [
    'square_full', # 0
    'square_low_rank', # 1
    'tall_full', # 2
    'tall_low_rank', # 3
    'fat_full', # 4
    'fat_low_rank', # 5
    'ill-conditioned',  # 6
    'identity'] # 7

mat = mat_list[6]
epsilon = 1e-8

if mat == 'square_full':
    A = np.array([[1.0, 2.0, 3.0, 4.0],
                  [4.0, 1.0, 0.0, -1.0],
                  [3.0, 5.0, -2.0, 1.0],
                  [2.0, 0.0, 1.0, 2.0]])

elif mat == 'square_low_rank':
    A = np.array([[1.0, 2.0, 3.0, 3.0],
                  [4.0, 1.0, 0.0, 0.0],
                  [3.0, 5.0, -2.0, -2.0],
                  [2.0, 0.0, 1.0, 1.0]])

elif mat == 'tall_full':
    A = np.array([[1.0, 2.0, 3.0],
                  [4.0, 1.0, 0.0],
                  [3.0, 5.0, -2.0],
                  [2.0, 0.0, 1.0],
                  [5.1, 5.2, 5.3]])

elif mat == 'tall_low_rank':
    A = np.array([[1.0, 1.0, 3.0],
                  [4.0, 4.0, 0.0],
                  [3.0, 3.0, -2.0],
                  [2.0, 2.0, 1.0],
                  [5.1, 5.1, 5.3]])

elif mat == 'fat_full':
    A = np.array([[1.0, 1.2, 1.7, 1.1, 5.7],
                  [4.0, 4.3, 4.1, 2.2, 6.6],
                  [3.0, 3.4, 3.9, 3.3, 5.2]])

elif mat == 'fat_low_rank':
    A = np.array([[1.0, 1.0, 1.0, 1.1, 5.7],
                  [4.0, 4.0, 4.0, 2.2, 6.6],
                  [3.0, 3.0, 3.0, 3.3, 5.2]])

elif mat == 'ill-conditioned':
    A = np.array([[1, 1, 1],
                  [epsilon, 0, 0],
                  [0, epsilon, 0],
                  [0, 0, epsilon]])

elif mat == 'identity':
    A = np.identity(4)

Q, R = np.linalg.qr(A)
print("Q from NumPy:\n", Q)
print("\nR from NumPy:\n", R)

Q, R = general_gram_schmidt(A, modified=True)

print("\nOrthonormal basis Q:")
print(Q)

print("\nUpper triangular matrix R:")
print(R)

# Verify Q is orthonormal
print(f"\nQ^TQ:\n{np.dot(Q.T, Q)}")
print(f"Norms: \n{np.linalg.norm(Q, axis=0)}")

A_reconstructed = np.dot(Q, R)
print("\nOriginal matrix A:")
print(A)
print("\nReconstructed matrix A from Q and R as QR:")
print(A_reconstructed)

print(f"\nReconstruction error from A_reconstructed:\n{np.linalg.norm(A - A_reconstructed)}")

Q from NumPy:
 [[-1.0000  0.0000  0.0000]
 [-0.0000 -0.7071 -0.4082]
 [-0.0000  0.7071 -0.4082]
 [-0.0000  0.0000  0.8165]]

R from NumPy:
 [[-1.0000 -1.0000 -1.0000]
 [ 0.0000  0.0000  0.0000]
 [ 0.0000  0.0000  0.0000]]

Orthonormal basis Q:
[[ 1.0000  0.0000  0.0000]
 [ 0.0000 -0.7071 -0.4082]
 [ 0.0000  0.7071 -0.4082]
 [ 0.0000  0.0000  0.8165]]

Upper triangular matrix R:
[[ 1.0000  1.0000  1.0000]
 [ 0.0000  0.0000  0.0000]
 [ 0.0000  0.0000  0.0000]]

Q^TQ:
[[ 1.0000 -0.0000 -0.0000]
 [-0.0000  1.0000  0.0000]
 [-0.0000  0.0000  1.0000]]
Norms: 
[ 1.0000  1.0000  1.0000]

Original matrix A:
[[ 1.0000  1.0000  1.0000]
 [ 0.0000  0.0000  0.0000]
 [ 0.0000  0.0000  0.0000]
 [ 0.0000  0.0000  0.0000]]

Reconstructed matrix A from Q and R as QR:
[[ 1.0000  1.0000  1.0000]
 [ 0.0000  0.0000  0.0000]
 [ 0.0000  0.0000 -0.0000]
 [ 0.0000  0.0000  0.0000]]

Reconstruction error from A_reconstructed:
2.479048431928874e-25


In [14]:
Q, R = general_gram_schmidt(A, modified=False)

print("Orthonormal basis Q:")
print(Q)

print("\nUpper triangular matrix R:")
print(R)

# Verify Q is orthonormal
print(f"\nQ^TQ:\n{np.dot(Q.T, Q)}")
print(f"Norms: \n{np.linalg.norm(Q, axis=0)}")

# Verify that A = QR
A_reconstructed = np.dot(Q, R)
print("\nOriginal matrix A:")
print(A)
print("\nReconstructed matrix A from Q and R:")
print(A_reconstructed)

Orthonormal basis Q:
[[ 1.0000  0.0000  0.0000]
 [ 0.0000 -0.7071 -0.7071]
 [ 0.0000  0.7071  0.0000]
 [ 0.0000  0.0000  0.7071]]

Upper triangular matrix R:
[[ 1.0000  1.0000  1.0000]
 [ 0.0000  0.0000  0.0000]
 [ 0.0000  0.0000  0.0000]]

Q^TQ:
[[ 1.0000 -0.0000 -0.0000]
 [-0.0000  1.0000  0.5000]
 [-0.0000  0.5000  1.0000]]
Norms: 
[ 1.0000  1.0000  1.0000]

Original matrix A:
[[ 1.0000  1.0000  1.0000]
 [ 0.0000  0.0000  0.0000]
 [ 0.0000  0.0000  0.0000]
 [ 0.0000  0.0000  0.0000]]

Reconstructed matrix A from Q and R:
[[ 1.0000  1.0000  1.0000]
 [ 0.0000  0.0000  0.0000]
 [ 0.0000  0.0000  0.0000]
 [ 0.0000  0.0000  0.0000]]


Here, we see that $Q^TQ$ is no longer producing identity matrix, indicating the CGS failed to find the orthonormal basis

However, MGS still produces orthonormal basis